In [ ]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

from MazeGenerationAlgorithms.LoopPrim import LoopPrimMaze
from MazeGenerationAlgorithms.LoopHuntAndKill import LoopHuntAndKillMaze

import pandas as pd
import networkx as nx

In [ ]:
class DataExporter:

    def __init__(self, maze_objects):
        self.mazes = maze_objects
        self.stats_df = self._analyze_mazes()

    def _analyze_mazes(self):
        stats = []

        for idx, maze in enumerate(self.mazes, start=1):  
            maze_type = maze.maze_name.replace(' Algorithm', '').strip()
            maze_size = f"{maze.width}x{maze.height}"
            loop_density = maze.loop_density

            solution_path = nx.shortest_path(maze.maze, maze.start, maze.end)
            solution_set = set(solution_path)

            sol_len = len(solution_path) - 1
            manhattan = abs(maze.end[0] - maze.start[0]) + abs(maze.end[1] - maze.start[1])
            tortuosity = sol_len / manhattan if manhattan > 0 else 1

            deadends = [n for n, d in maze.maze.degree() if d == 1 and n not in [maze.start, maze.end]]
            intersections = [n for n, d in maze.maze.degree() if d > 2]

            deadends = [n for n, d in maze.maze.degree() if d == 1 and n not in [maze.start, maze.end]]
            intersections = [n for n, d in maze.maze.degree() if d > 2]
            deadend_crossroads = self._count_deadend_crossroads(maze.maze, deadends, intersections)

            dist_from_solution = self._get_node_to_deadend_distances(maze.maze, solution_set, deadends)
            dist_from_intersections = self._get_node_to_deadend_distances(maze.maze, intersections, deadends)

            avg_sol_deadend = self._average_distance(dist_from_solution)
            max_sol_deadend = self._maximum_distance(dist_from_solution)
            avg_inter_deadend = self._average_distance(dist_from_intersections)

            tempting_count = self._tempting_count(maze, solution_path)

            stats.append({
                'Maze Type': maze_type,
                'Size': maze_size,
                'Loop Density': loop_density,
                '# of Intersections': len(intersections),
                'Solution Length': sol_len,
                'Shortest Possible Solution': manhattan,
                'Solution Path Tortuosity': round(tortuosity, 2),
                '# of Deadends': len(deadends),
                'Deadend Crossroads': deadend_crossroads,
                'Avg Distance to Deadend(From Solution Path)': avg_sol_deadend,
                'Max Distance to Deadend(From Solution Path)': max_sol_deadend,
                'Avg Distance to Deadend(From Intersection)': avg_inter_deadend,
                'Tempting Count': tempting_count
            })

        return pd.DataFrame(stats)

    def _tempting_count(self, maze, path):
        maze_graph = maze.maze
        tempting = 0

        for i in range(1, len(path) - 1):
            prev = path[i - 1]
            curr = path[i]

            forward_choices = sum(1 for neighbor in maze_graph.neighbors(curr) if neighbor != prev)

            if forward_choices > 1:
                tempting += forward_choices - 1

        return tempting

    def _get_node_to_deadend_distances(self, graph, nodes, deadends):
        if not nodes or not deadends:
            return {}
        distances = {}
        for src in nodes:
            for tgt in deadends:
                try:
                    dist = len(nx.shortest_path(graph, src, tgt)) - 1
                    distances[f"{src}-{tgt}"] = dist
                except nx.NetworkXNoPath:
                    continue
        return distances

    def _average_distance(self, dist_map):
        return round(sum(dist_map.values()) / len(dist_map), 2) if dist_map else 0

    def _maximum_distance(self, dist_map):
        return max(dist_map.values(), default=0)
    
    def _count_deadend_crossroads(self, graph, deadends, intersections):
        count = 0
        for deadend in deadends:
            neighbors = list(graph.neighbors(deadend))
            if neighbors and neighbors[0] in intersections:
                count += 1
        return count

In [ ]:
width, height = 25, 25
sample_size = 100
loop_densities = [0, 0.1, 0.25]
all_data = []

for density in loop_densities:
    maze_lst = []
    for _ in range(sample_size):
        maze_lst.extend([
            LoopHuntAndKillMaze(width, height, loop_density=density),
            LoopPrimMaze(width, height, loop_density=density),
        ])
    data = DataExporter(maze_lst)
    all_data.append(data.stats_df)

final_data = pd.concat(all_data, ignore_index=True)
final_data.to_csv('loop_metric.csv', index=False)